#### Q1. build.version_hash = 42f05b9372a9a4a470db3b52817899b99a76ee73

In [1]:
pip install requests


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

### Q2. Indexing data

In [3]:
from elasticsearch import Elasticsearch

In [4]:
es_client = Elasticsearch('http://localhost:9200')

In [5]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

#### Define schema

In [6]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0,
    },
    "mappings": {
        "properties": {
            "question": {"type": "text"},
            "section": {"type": "text"},
            "text": {"type": "text"},
            "course": {"type": "keyword"}
        }
    }
}

index_name = 'course-questions'

es_client.indices.create(index=index_name, body=index_settings)

/tmp/ipykernel_48606/4073002207.py:18: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es_client.indices.create(index=index_name, body=index_settings)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

#### Populate ES table

In [7]:
from tqdm.auto import tqdm

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:03<00:00, 303.31it/s]


### Q3. Searching

In [9]:
query = "How do execute a command on a Kubernetes pod?"

In [10]:
search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }  
            }
        }
    }
}

search_response = es_client.search(index=index_name, body=search_query)

/tmp/ipykernel_48606/1991316880.py:16: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  search_response = es_client.search(index=index_name, body=search_query)


In [11]:
search_response['hits']['max_score']

44.06996

### Q4. Filtering

In [12]:
query_2 = "How do copy a file to a Docker container?"

In [13]:
search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }  
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

search_response = es_client.search(index=index_name, body=search_query)

/tmp/ipykernel_48606/2547665400.py:21: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  search_response = es_client.search(index=index_name, body=search_query)


In [14]:
relevant_documents = []

for doc in search_response['hits']['hits']:
    relevant_documents.append(doc['_source'])

In [15]:
relevant_documents[2]['question']

'How do I copy files from a different folder into docker container’s working directory?'

### Q5. Building a prompt

In [16]:
context_template = """
Q: {question}
A: {text}
""".strip()

In [17]:
context = ""

for doc in relevant_documents:
    question = doc['question']
    text = doc['text']
    context += context_template.format(question=question, text=text) + "\n\n"

In [18]:
query_2

'How do copy a file to a Docker container?'

In [19]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [20]:
prompt = prompt_template.format(question=query_2, context=context)

In [21]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do copy a file to a Docker container?

CONTEXT:
Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: Kubernetes-dashboard
A: Deploy and Access the Kubernetes Dashboard
Luke

Q: How do I copy files from a different folder into docker container’s working directory?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
In the Dockerfile, you can provide the folder containing the files that you want to copy over. The basic syntax is as follows:

In [22]:
len(prompt)

1919

### Q6. Tokens - For OpenAI API question only, I used Gemini :<

### Bonus: Gemini Answer

In [23]:
import os
gemini_api_key = os.environ['GEMINI_API_KEY']

In [24]:
from google import genai
llm_client = genai.Client(api_key=gemini_api_key)

In [25]:
response = llm_client.models.generate_content(
    model='gemini-2.0-flash',
    contents=prompt
)

In [26]:
answer = response.candidates[0].content.parts[0].text

In [27]:
answer

'You can copy files from your local machine into a Docker container using the `docker cp` command. In the Dockerfile, you can provide the folder containing the files that you want to copy over, using the syntax: `COPY ["src/predict.py", "models/xgb_model.bin", "./"]`.\n'